In [1]:
import numpy as np
from collections import defaultdict, Counter
from math import log
import string


class SpamNB:
    def __init__(self):
        self.classes = defaultdict(int)
        self.freqs = defaultdict(int)
        
    def __extract_features(self, feature):
        if len(feature) != 0:
            for c in string.punctuation:
                feature = feature.replace(c, '')
            feature = feature.lower()
            feature = np.array(feature.split(' '))
            return feature
        else:
            return []
        
    def fit(self, X, y):
        self.__init__()
        if (len(X) != len(y)):
            raise
            print('X and y must have the same length')
        # Определим классы, посчитаем количество объектов в классе
        self.classes = Counter(y)
        # Посчитаем фичи.         
        for i in range(len(y)):
            item = X[i]
            label = y[i]
            feats = self.__extract_features(item)
            # Для каждого из объектов суммируем фичи с учетом класса
            for feat in feats:
                self.freqs[(label, feat)] += 1
        # Отнормируем
        for label, feat in self.freqs:
            self.freqs[(label, feat)] /= self.classes[label]
        for label in self.classes:
            self.classes[label] /= len(y)

        return self
    
    def predict(self, X):
        y = np.zeros((len(X),))
        for i in range(len(X)):
            item = X[i]
            feats = self.__extract_features(item)
            keys = self.classes.keys()
            label = min(
                self.classes.keys(), 
                key=lambda c: (
                    -log(self.classes[c]) 
                    + sum(-log(self.freqs.get((c, feat), 10**(-5))) for feat in feats)
                )
            )
            y[i] = label
        return y
    

In [2]:
NB = SpamNB()
x = np.array(['Путевки по низкой цене', 'Акция! Купи шоколадку и получи телефон в подарок', 'Завтра состоится собрание', 'Купи килограмм яблок и шоколадку'])
y = np.array([1, 1 ,0, 0])
x_test = np.array(['В магазине гора яблок. Купи семь килограмм и шоколадку', 'Акция!!! Бухгалтерия по низким ценам уже завтра'])
NB.fit(x, y)
NB.predict(x_test)

array([0., 1.])